In [15]:
import math
import re
import os

import pandas as pd

In [16]:
DATA_DIR = '../data'
WEIGHTS_FILE = f"{DATA_DIR}/weights/weights.csv"

In [17]:
df = pd.read_csv(WEIGHTS_FILE)
df = df.dropna()
df.head(3)

,Tag,Count,Key,Value,Weight
0,"('highway', 'footway')",6913,highway,footway,100.0
1,"('footway', 'sidewalk')",4337,footway,sidewalk,100.0
2,"('crossing', 'traffic_signals')",628,crossing,traffic_signals,100.0


In [18]:
grouped_df = df.groupby('Key')[['Value', 'Weight']].agg(list).reset_index()
grouped_df.head(10)

,Key,Value,Weight
0,crossing,"[traffic_signals, marked]","[100.0, 1.0]"
1,footway,[sidewalk],[100.0]
2,highway,[footway],[100.0]


In [19]:
get_tags = ""
tag_conditions = ""

for _, row in grouped_df.iterrows():
    get_tag = f"""
  local {row['Key']} = way:get_value_by_key('{row['Key']}')
    """
    get_tags += get_tag
    for value, weight in zip(row['Value'], row['Weight']):
        if weight == 1:
            continue
        
        if tag_conditions == "":
            tag_condition = f"""
  if {row['Key']} == '{value}' then
"""
        else:
            tag_condition = f"""
  elseif {row['Key']} == '{value}' then
"""
        
        tag_condition += f"""
     result.forward_speed = walking_speed * {weight}
     result.backward_speed = walking_speed * {weight}
        """
        tag_conditions += tag_condition

tag_conditions += f"""
  else
    result.forward_speed = walking_speed
    result.backward_speed = walking_speed
  end
"""

custom_tag_weigths = f"""
function custom_tag_weights(profile, way, result, data)
{get_tags}
{tag_conditions}
end
"""

custom_tag_weigths = re.sub(r'\n\s*\n', '\n', custom_tag_weigths)

In [20]:
osrm_profile = f"""
-- Foot profile

api_version = 2

Set = require('lib/set')
Sequence = require('lib/sequence')
Handlers = require('lib/way_handlers')
find_access_tag = require('lib/access').find_access_tag

function setup()
walking_speed = 5
  return {{
    properties = {{
      weight_name                   = 'duration',
      max_speed_for_map_matching    = 40/3.6, -- kmph -> m/s
      call_tagless_node_function    = false,
      traffic_light_penalty         = 2,
      u_turn_penalty                = 2,
      continue_straight_at_waypoint = false,
      use_turn_restrictions         = false,
    }},

    default_mode            = mode.walking,
    default_speed           = walking_speed,
    oneway_handling         = 'specific',     -- respect 'oneway:foot' but not 'oneway'

    barrier_blacklist = Set {{
      'yes',
      'wall',
      'fence'
    }},

    access_tag_whitelist = Set {{
      'yes',
      'foot',
      'permissive',
      'designated'
    }},

    access_tag_blacklist = Set {{
      'no',
      'agricultural',
      'forestry',
      'private',
      'delivery',
    }},

    restricted_access_tag_list = Set {{ }},

    restricted_highway_whitelist = Set {{ }},

    construction_whitelist = Set {{}},

    access_tags_hierarchy = Sequence {{
      'foot',
      'access'
    }},

    -- tags disallow access to in combination with highway=service
    service_access_tag_blacklist = Set {{ }},

    restrictions = Sequence {{
      'foot'
    }},

    -- list of suffixes to suppress in name change instructions
    suffix_list = Set {{
      'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'North', 'South', 'West', 'East'
    }},

    avoid = Set {{
      'impassable',
      'proposed'
    }},

    speeds = Sequence {{
      highway = {{
        primary         = walking_speed,
        primary_link    = walking_speed,
        secondary       = walking_speed,
        secondary_link  = walking_speed,
        tertiary        = walking_speed,
        tertiary_link   = walking_speed,
        unclassified    = walking_speed,
        residential     = walking_speed,
        road            = walking_speed,
        living_street   = walking_speed,
        service         = walking_speed,
        track           = walking_speed,
        path            = walking_speed,
        steps           = walking_speed,
        pedestrian      = walking_speed,
        footway         = walking_speed,
        pier            = walking_speed,
      }},

      railway = {{
        platform        = walking_speed
      }},

      amenity = {{
        parking         = walking_speed,
        parking_entrance= walking_speed
      }},

      man_made = {{
        pier            = walking_speed
      }},

      leisure = {{
        track           = walking_speed
      }}
    }},

    route_speeds = {{
      ferry = 5
    }},

    bridge_speeds = {{
    }},

    surface_speeds = {{
      fine_gravel =   walking_speed*0.75,
      gravel =        walking_speed*0.75,
      pebblestone =   walking_speed*0.75,
      mud =           walking_speed*0.5,
      sand =          walking_speed*0.5
    }},

    tracktype_speeds = {{
    }},

    smoothness_speeds = {{
    }}
  }}
end

function process_node(profile, node, result)
  -- parse access and barrier tags
  local access = find_access_tag(node, profile.access_tags_hierarchy)
  if access then
    if profile.access_tag_blacklist[access] then
      result.barrier = true
    end
  else
    local barrier = node:get_value_by_key('barrier')
    if barrier then
      --  make an exception for rising bollard barriers
      local bollard = node:get_value_by_key('bollard')
      local rising_bollard = bollard and 'rising' == bollard

      if profile.barrier_blacklist[barrier] and not rising_bollard then
        result.barrier = true
      end
    end
  end

  -- check if node is a traffic light
  local tag = node:get_value_by_key('highway')
  if 'traffic_signals' == tag then
    -- Direction should only apply to vehicles
    result.traffic_lights = true
  end
end

-- main entry point for processsing a way
function process_way(profile, way, result)
  -- the intial filtering of ways based on presence of tags
  -- affects processing times significantly, because all ways
  -- have to be checked.
  -- to increase performance, prefetching and intial tag check
  -- is done in directly instead of via a handler.

  -- in general we should  try to abort as soon as
  -- possible if the way is not routable, to avoid doing
  -- unnecessary work. this implies we should check things that
  -- commonly forbids access early, and handle edge cases later.

  -- data table for storing intermediate values during processing
  local data = {{
    -- prefetch tags
    highway = way:get_value_by_key('highway'),
    bridge = way:get_value_by_key('bridge'),
    route = way:get_value_by_key('route'),
    leisure = way:get_value_by_key('leisure'),
    man_made = way:get_value_by_key('man_made'),
    railway = way:get_value_by_key('railway'),
    platform = way:get_value_by_key('platform'),
    amenity = way:get_value_by_key('amenity'),
    public_transport = way:get_value_by_key('public_transport')
  }}

  -- perform an quick initial check and abort if the way is
  -- obviously not routable. here we require at least one
  -- of the prefetched tags to be present, ie. the data table
  -- cannot be empty
  if next(data) == nil then     -- is the data table empty?
    return
  end

  local handlers = Sequence {{
    -- set the default mode for this profile. if can be changed later
    -- in case it turns we're e.g. on a ferry
    WayHandlers.default_mode,

    -- check various tags that could indicate that the way is not
    -- routable. this includes things like status=impassable,
    -- toll=yes and oneway=reversible
    WayHandlers.blocked_ways,

    -- determine access status by checking our hierarchy of
    -- access tags, e.g: motorcar, motor_vehicle, vehicle
    WayHandlers.access,

    -- check whether forward/backward directons are routable
    WayHandlers.oneway,

    -- check whether forward/backward directons are routable
    WayHandlers.destinations,

    -- check whether we're using a special transport mode
    WayHandlers.ferries,
    WayHandlers.movables,

    -- compute speed taking into account way type, maxspeed tags, etc.
    WayHandlers.speed,
    WayHandlers.surface,

    -- handle turn lanes and road classification, used for guidance
    WayHandlers.classification,

    -- handle various other flags
    WayHandlers.roundabouts,
    WayHandlers.startpoint,

    -- set name, ref and pronunciation
    WayHandlers.names,

    -- set weight properties of the way
    WayHandlers.weights,

    custom_tag_weights
  }}

  WayHandlers.run(profile, way, result, data, handlers)
end

function process_turn (profile, turn)
  turn.duration = 0.

  if turn.direction_modifier == direction_modifier.u_turn then
     turn.duration = turn.duration + profile.properties.u_turn_penalty
  end

  if turn.has_traffic_light then
     turn.duration = profile.properties.traffic_light_penalty
  end
  if profile.properties.weight_name == 'routability' then
      -- penalize turns from non-local access only segments onto local access only tags
      if not turn.source_restricted and turn.target_restricted then
          turn.weight = turn.weight + 3000
      end
  end
end

{custom_tag_weigths}

return {{
  setup = setup,
  process_way =  process_way,
  process_node = process_node,
  process_turn = process_turn
}}
"""

In [21]:
with open("../profiles/custom.lua", 'w') as file:
    file.write(osrm_profile)